In [ ]:
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

In [ ]:
df = pd.read_csv('./air_quality_no2_long.csv')

# 날짜 및 시간 처리
df['date.utc'] = pd.to_datetime(df['date.utc'])
df['year'] = df['date.utc'].dt.year
df['month'] = df['date.utc'].dt.month
df['day'] = df['date.utc'].dt.day
df['hour'] = df['date.utc'].dt.hour

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2068 entries, 0 to 2067
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype              
---  ------     --------------  -----              
 0   city       2068 non-null   object             
 1   country    2068 non-null   object             
 2   date.utc   2068 non-null   datetime64[ns, UTC]
 3   location   2068 non-null   object             
 4   parameter  2068 non-null   object             
 5   value      2068 non-null   float64            
 6   unit       2068 non-null   object             
 7   year       2068 non-null   int64              
 8   month      2068 non-null   int64              
 9   day        2068 non-null   int64              
 10  hour       2068 non-null   int64              
dtypes: datetime64[ns, UTC](1), float64(1), int64(4), object(5)
memory usage: 177.8+ KB


In [ ]:
df['unit'].value_counts()

0    2068
Name: unit, dtype: int64

In [ ]:
df2 = df.drop(columns=['parameter', 'unit', 'date.utc'])
df2

,city,country,location,value,year,month,day,hour
0,Paris,FR,FR04014,20.0,2019,6,21,0
1,Paris,FR,FR04014,21.8,2019,6,20,23
2,Paris,FR,FR04014,26.5,2019,6,20,22
3,Paris,FR,FR04014,24.9,2019,6,20,21
4,Paris,FR,FR04014,21.4,2019,6,20,20
...,...,...,...,...,...,...,...,...
2063,London,GB,London Westminster,26.0,2019,5,7,6
2064,London,GB,London Westminster,16.0,2019,5,7,4
2065,London,GB,London Westminster,19.0,2019,5,7,3
2066,London,GB,London Westminster,19.0,2019,5,7,2


In [ ]:
label_encoder = LabelEncoder()
df['country'] = label_encoder.fit_transform(df['country'])

In [ ]:
x = df.drop(columns=['country', 'date.utc', 'parameter','unit']) # 'city', 'location'
y = df['country']

In [ ]:
x.head()

,city,location,value,year,month,day,hour
0,Paris,FR04014,20.0,2019,6,21,0
1,Paris,FR04014,21.8,2019,6,20,23
2,Paris,FR04014,26.5,2019,6,20,22
3,Paris,FR04014,24.9,2019,6,20,21
4,Paris,FR04014,21.4,2019,6,20,20


In [ ]:
y = pd.get_dummies(y)

In [ ]:
y.head()

,0,1,2
0,0,1,0
1,0,1,0
2,0,1,0
3,0,1,0
4,0,1,0


In [ ]:
x = pd.get_dummies(x)
x.head()

,value,year,month,day,hour,city_Antwerpen,city_London,city_Paris,location_BETR801,location_FR04014,location_London Westminster
0,20.0,2019,6,21,0,0,0,1,0,1,0
1,21.8,2019,6,20,23,0,0,1,0,1,0
2,26.5,2019,6,20,22,0,0,1,0,1,0
3,24.9,2019,6,20,21,0,0,1,0,1,0
4,21.4,2019,6,20,20,0,0,1,0,1,0


In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense

def build_model():
    model = Sequential()
    model.add(Dense(32, input_dim=len(x.columns), activation='relu'))  # 입력층 (특성 수에 맞게 조정)
    model.add(Dense(16, activation='relu'))                # 은닉층
    model.add(Dense(3 , activation='softmax'))               # 출력층

    return model

model = build_model()

In [ ]:
model = build_model()
model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_38 (Dense)            (None, 32)                384       
                                                                 
 dense_39 (Dense)            (None, 16)                528       
                                                                 
 dense_40 (Dense)            (None, 3)                 51        
                                                                 
Total params: 963 (3.76 KB)
Trainable params: 963 (3.76 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [ ]:
history = model.fit(x, y, epochs=500, batch_size=16, validation_split=0.2,verbose=1)

Epoch 1/500
104/104 [==============================] - 1s 5ms/step - loss: 40.8622 - acc: 0.4492 - val_loss: 0.1880 - val_acc: 1.0000
Epoch 2/500
104/104 [==============================] - 0s 2ms/step - loss: 1.4597 - acc: 0.5532 - val_loss: 1.2152 - val_acc: 0.0217
Epoch 3/500
104/104 [==============================] - 0s 2ms/step - loss: 1.2210 - acc: 0.5877 - val_loss: 2.2416 - val_acc: 0.0000e+00
Epoch 4/500
104/104 [==============================] - 0s 3ms/step - loss: 1.0481 - acc: 0.6511 - val_loss: 2.5220 - val_acc: 0.0000e+00
Epoch 5/500
104/104 [==============================] - 0s 3ms/step - loss: 0.9760 - acc: 0.7080 - val_loss: 1.1684 - val_acc: 0.0217
Epoch 6/500
104/104 [==============================] - 0s 3ms/step - loss: 1.1012 - acc: 0.6360 - val_loss: 0.0602 - val_acc: 0.9976
Epoch 7/500
104/104 [==============================] - 0s 3ms/step - loss: 0.9192 - acc: 0.6844 - val_loss: 0.2631 - val_acc: 0.9710
Epoch 8/500
104/104 [==============================] - 0s 4m

KeyboardInterrupt: ignored

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,random_state=2000, test_size=0.2)

In [ ]:
# RandomForest
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
model.fit(x_train,y_train)
model.score(x_test,y_test)

0.7391304347826086

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

df = pd.read_csv('air_quality_no2_long.csv')
df['value'].describe()
df['air_quality'] = pd.cut(df['value'], bins=[-float('inf'), 30, 60, float('inf')], labels=[0, 1, 2], right=False).astype(int)
df.isnull().sum()
df.info()

df['date.utc'] = pd.to_datetime(df['date.utc'])

# 각 구성 요소를 One-Hot 인코딩
df = pd.concat([df, df['date.utc'].dt.year.rename('year'),
                df['date.utc'].dt.month.rename('month'),
                df['date.utc'].dt.day.rename('day'),
                df['date.utc'].dt.hour.rename('hour'),
                df['date.utc'].dt.minute.rename('minute'),
                df['date.utc'].dt.second.rename('second'),
                pd.get_dummies(df['date.utc'].dt.tz, prefix='timezone')], axis=1)

# 'date.utc' 열 삭제
df = df.drop(columns=['date.utc', 'timezone_UTC'])

col= df.select_dtypes('object').columns.values
print(col)

df = pd.get_dummies(df, columns=col)

X_data = df.drop(['air_quality'], axis=1)
y_data = df['air_quality']

y_data.value_counts()

from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=42)

print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

#y_train_int = LabelEncoder().fit_transform(y_train)

y_train_onehot = to_categorical(y_train, num_classes=3)
y_test_onehot = to_categorical(y_test, num_classes=3)

from tensorflow import keras
from tensorflow.keras import layers

def build_model():
  model = keras.Sequential()
  model.add(layers.Dense(36,input_dim=18, activation='relu'))
  model.add(layers.Dense(24, activation='relu'))
  model.add(layers.Dense(12, activation='relu'))
  model.add(layers.Dense(6, activation='relu'))
  model.add(layers.Dense(3, activation='softmax'), )
  return model

model = build_model()
model.summary()

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

history = model.fit(X_train, y_train_onehot, epochs=100, batch_size=16, validation_data=(X_test, y_test_onehot))
# 학습곡선 그리기

import matplotlib.pyplot as plt

def plot_history(history):
  hist=pd.DataFrame(history.history)
  hist['epoch']=history.epoch

  plt.figure(figsize=(16,8))
  plt.subplot(1,2,1)
  plt.xlabel('Epoch')
  plt.ylabel('Loss')
  plt.plot(hist['epoch'],hist['loss'],label='Train Loss')
  plt.plot(hist['epoch'], hist['val_loss'],label='Val Loss')

  plt.subplot(1,2,1)
  plt.xlabel('Epoch')
  plt.ylabel('Accuracy')
  plt.plot(hist['epoch'],hist['acc'],label='Train Accuracy')
  plt.plot(hist['epoch'], hist['val_acc'],label='Val Accuracy')
  plt.legend()

  plt.show()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2068 entries, 0 to 2067
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   city         2068 non-null   object 
 1   country      2068 non-null   object 
 2   date.utc     2068 non-null   object 
 3   location     2068 non-null   object 
 4   parameter    2068 non-null   object 
 5   value        2068 non-null   float64
 6   unit         2068 non-null   object 
 7   air_quality  2068 non-null   int64  
dtypes: float64(1), int64(1), object(6)
memory usage: 129.4+ KB
['city' 'country' 'location' 'parameter' 'unit']
(1654, 18) (414, 18)
(1654,) (414,)
Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_24 (Dense)            (None, 36)                684       
                                                                 
 dense_25 (Dense)            (None, 24)